# Define BiKA Linear Layer

In [1]:
import math
import torch
import torch.nn as nn
from torch.nn import Module
import torch.nn.functional as F

In [2]:
class CustomSignFunction(torch.autograd.Function):
    @staticmethod
    def forward(ctx, input):
        # Save the input for backward computation
        ctx.save_for_backward(input)
        # Output +1 for input > 0, else -1 (including for input == 0)
        return torch.where(input > 0, torch.tensor(1.0, device=input.device), torch.tensor(-1.0, device=input.device))

    @staticmethod
    def backward(ctx, grad_output):
        # Retrieve the input saved in the forward pass
        input, = ctx.saved_tensors
        # Gradient of the input is the same as the gradient output (STE)
        grad_input = grad_output.clone()
        # Pass the gradient only where input was non-zero, otherwise set it to 0
        grad_input[input.abs() > 0] = grad_output[input.abs() > 0]
        return grad_input

# Wrapper class for convenience
class CustomSignActivation(torch.nn.Module):
    def __init__(self):
        super(CustomSignActivation, self).__init__()

    def forward(self, input):
        return CustomSignFunction.apply(input)

In [3]:
class BiKALinear(nn.Module):
    def __init__(self, in_features, out_features):
        super(BiKALinear, self).__init__()
        
        self.in_features = in_features
        self.out_features = out_features
        self.weight = nn.Parameter(torch.Tensor(out_features, in_features))
        self.bias = nn.Parameter(torch.Tensor(out_features, in_features))
        self.sign = CustomSignActivation()
            
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.kaiming_uniform_(self.weight, a=math.sqrt(5))
        fan_in, _ = nn.init._calculate_fan_in_and_fan_out(self.weight)
        bound = 1 / math.sqrt(fan_in)
        nn.init.uniform_(self.bias, -bound, bound)

    def forward(self, x):
        # Expand the input to match the bias shape for broadcasting
        # x is of shape (batch_size, in_features)
        # Expand bias matrix to (batch_size, out_features, in_features)
        x = x.unsqueeze(1) + self.bias.unsqueeze(0)
        
        # Perform element-wise multiplication with weights
        x = x * self.weight.unsqueeze(0)
        
        # Apply sign function: -1 for negative and 0, 1 for positive
        x = self.sign(x)
        
        # Sum the thresholded products along the input features dimension
        x = torch.sum(x, dim=-1) 

        return x

# Try MLP-like BiKA with MNIST

## 1. Dataset Loading

In [4]:
import torchvision
from torchvision import datasets
from torchvision import transforms
from torch.autograd import Variable
from torch.utils.data import DataLoader, random_split

In [5]:
full_data_train = torchvision.datasets.MNIST('./data/', 
                                        train=True, download=True,
                                        transform=torchvision.transforms.Compose
                                        ([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.5,), (0.5,))
                                        ]))

# Split the dataset into training and validation subsets
train_size = int(0.8 * len(full_data_train))
val_size = len(full_data_train) - train_size
data_train, data_valid = random_split(full_data_train, [train_size, val_size])

data_test = torchvision.datasets.MNIST('./data/', 
                                       train=False, download=True,
                                       transform=torchvision.transforms.Compose
                                       ([
                                            torchvision.transforms.ToTensor(),
                                            torchvision.transforms.Normalize((0.5,), (0.5,))
                                       ]))

## 2. Define MLP-like BiKA structure

In [6]:
import math
import torch
import torch.nn as nn
from torch.nn import Module
import torch.nn.functional as F
import torch.optim as optim
from sklearn.metrics import accuracy_score
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm, trange

In [7]:
import brevitas.nn as qnn
from brevitas.nn import QuantLinear, QuantReLU, QuantConv2d
from brevitas.quant.binary import SignedBinaryActPerTensorConst
from brevitas.quant.binary import SignedBinaryWeightPerTensorConst
from brevitas.inject.enum import QuantType

In [8]:
input_size = 1*28*28      
hidden1 = 512      
hidden2 = 512
hidden3 = 512
num_classes = 10  

In [9]:
class BiKA_MNIST(Module):
    def __init__(self):
        super(BiKA_MNIST, self).__init__()
        
        self.fc0   = BiKALinear(in_features=input_size, out_features=hidden1)
        
        self.fc1   = BiKALinear(in_features=hidden1, out_features=hidden2)
        
        self.fc2   = BiKALinear(in_features=hidden2, out_features=hidden3)
        
        self.out   = BiKALinear(in_features=hidden3, out_features=num_classes)

    def forward(self, x):
        
        out = x.reshape(x.shape[0], -1)
        
        out = self.fc0(out)
        out = self.fc1(out)
        out = self.fc2(out)
        out = self.out(out)
        
        return out

## 3. Define Training Function

In [10]:
num_of_gpus = torch.cuda.device_count()
print(num_of_gpus)

# Check for GPU
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Setting seeds for reproducibility
torch.manual_seed(0)

2
Using device: cuda:0


In [11]:
def display_loss_plot(losses, title="Training loss", xlabel="Iterations", ylabel="Loss"):
    x_axis = [i for i in range(len(losses))]
    plt.plot(x_axis,losses)
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.show()

In [12]:
def train_and_validate(model, train_loader, val_loader, criterion, learning_rate):
    model.train()
    
    optimizer = optim.Adam(model.parameters(), lr=learning_rate[0])
    
    for epoch in range(100):
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            # Backward pass
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            
        # Adjust learning rate at epoch 100
        if epoch+1 == 50:
            for param_group in optimizer.param_groups:
                param_group['lr'] = learning_rate[1]
                print(f"Learning rate changed to {param_group['lr']} at epoch {epoch+1}")
        
        # Adjust learning rate at epoch 150
        if epoch+1 == 75:
            for param_group in optimizer.param_groups:
                param_group['lr'] = learning_rate[2]
                print(f"Learning rate changed to {param_group['lr']} at epoch {epoch+1}")

        # Validation phase
        model.eval()
        all_preds = []
        all_labels = []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                _, preds = torch.max(outputs, 1)
                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

        val_acc = accuracy_score(all_labels, all_preds)
        print(f"Epoch [{epoch+1}/{100}], "
              f"Train Loss: {running_loss/len(train_loader):.4f}, "
              f"Val Accuracy: {val_acc*100:.2f}%")
        
    return val_acc

## 4. Define Evaluation Function

In [13]:
def evaluate_model(model, test_loader):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
    test_acc = accuracy_score(all_labels, all_preds)
    print(f"Test Accuracy: {test_acc * 100:.2f}%")
    return test_acc

## 5. Train MLP-like BiKA for MNIST

In [14]:
batch_sizes = [64, 128, 256]
learning_rates = [[0.0100, 0.0010, 0.0010],
                  [0.0010, 0.0010, 0.0010],
                  [0.0010, 0.0010, 0.0001],
                  [0.0010, 0.0005, 0.0001]
                 ]

In [15]:
best_acc = 0.0
best_params = None

for batch_size, learning_rate in product(batch_sizes, learning_rates):
    print(f"Training with batch_size={batch_size}, learning_rate={learning_rate}")

    # Data loaders
    train_loader = DataLoader(data_train, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(data_valid, batch_size=batch_size, shuffle=False)

    # Initialize the model, loss, and optimizer
    model = BiKA_MNIST().to(device)
    criterion = nn.CrossEntropyLoss()

    # Train and validate
    val_acc = train_and_validate(model, train_loader, val_loader, criterion, learning_rate)

    # Update best parameters
    if val_acc > best_acc:
        best_acc = val_acc
        best_params = (batch_size, learning_rate)

print(f"Best Accuracy: {best_acc*100:.2f}%")
print(f"Best Parameters: Batch Size={best_params[0]}, Learning Rate={best_params[1]}")

Training with batch_size=64, learning_rate=[0.01, 0.001, 0.001]


Epoch [1/100], Train Loss: 13.8142, Val Accuracy: 78.71%


Epoch [2/100], Train Loss: 22.6110, Val Accuracy: 73.86%


Epoch [3/100], Train Loss: 6.1351, Val Accuracy: 80.47%


Epoch [4/100], Train Loss: 18.5524, Val Accuracy: 54.77%


...

Epoch [96/100], Train Loss: 2.7502, Val Accuracy: 84.14%


Epoch [97/100], Train Loss: 2.7786, Val Accuracy: 85.05%


Epoch [98/100], Train Loss: 2.7462, Val Accuracy: 85.04%


Epoch [99/100], Train Loss: 2.8098, Val Accuracy: 84.82%


Epoch [100/100], Train Loss: 2.6606, Val Accuracy: 84.05%
Best Accuracy: 84.05%
Best Parameters: Batch Size=256, Learning Rate=[0.001, 0.0005, 0.0001]


## 6. Evaluate MLP-like BiKA for MNIST

In [16]:
train_loader = DataLoader(data_train, batch_size=best_params[0], shuffle=True)
val_loader = DataLoader(data_valid, batch_size=best_params[0], shuffle=False)
test_loader = DataLoader(data_test, batch_size=best_params[0], shuffle=False)

model = BiKA_MNIST().to(device)
criterion = nn.CrossEntropyLoss()

train_and_validate(model, train_loader, val_loader, criterion, best_params[1])

Epoch [1/100], Train Loss: 7.3992, Val Accuracy: 80.11%


Epoch [2/100], Train Loss: 5.1446, Val Accuracy: 81.33%


Epoch [3/100], Train Loss: 4.7571, Val Accuracy: 78.72%


Epoch [4/100], Train Loss: 4.6262, Val Accuracy: 81.63%


Epoch [5/100], Train Loss: 4.4450, Val Accuracy: 83.53%


...

Epoch [97/100], Train Loss: 2.9850, Val Accuracy: 83.23%


Epoch [98/100], Train Loss: 3.0466, Val Accuracy: 83.84%


Epoch [99/100], Train Loss: 3.0488, Val Accuracy: 82.33%


Epoch [100/100], Train Loss: 3.0018, Val Accuracy: 82.70%


0.827

In [17]:
print(f"Best Validation Accuracy: {best_acc*100:.2f}%")
print(f"Best Parameters: Batch Size={best_params[0]}, Learning Rate={best_params[1]}")

evaluate_model(model, test_loader)

Best Validation Accuracy: 84.05%
Best Parameters: Batch Size=256, Learning Rate=[0.001, 0.0005, 0.0001]


Test Accuracy: 82.94%


0.8294